In [92]:
from google.colab import files
files.upload()

Saving IIT Tp Assignment.xlsx to IIT Tp Assignment (9).xlsx


{'IIT Tp Assignment (9).xlsx': b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00\xadG\xa3/\x7f\x01\x00\x00\xc2\x05\x00\x00\x13\x00\x08\x02[Content_Types].xml \xa2\x04\x02(\xa0\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [94]:
!pip install pulp openpyxl -q

import math
import pandas as pd
import pulp


In [95]:
file_path = "/content/IIT Tp Assignment.xlsx"  # change if needed

q1 = pd.read_excel(file_path, sheet_name="Question 1")
display(q1.head())

I = list(q1.index)   # 0..49
x = {i: q1.loc[i, "x"] for i in I}
y = {i: q1.loc[i, "y"] for i in I}

# Depot
x0, y0 = 40, 40

# Build nodes: depot + customers
N = ["depot"] + I
X = {"depot": x0, **{i: x[i] for i in I}}
Y = {"depot": y0, **{i: y[i] for i in I}}

# Distances
d = {}
for i in N:
    for j in N:
        if i == j:
            d[(i, j)] = 0.0
        else:
            d[(i, j)] = math.hypot(X[i] - X[j], Y[i] - Y[j])

# Demand
q = {i: 1.5 for i in I}


,x,y,m
0,48,47,1
1,44,46,1
2,38,49,1
3,43,32,1
4,48,34,1


In [96]:
vehicles = []
vehicle_type = {}
Q = {}
F = {}
c = {}
speed = {}

# 4 bicycles
for k in range(4):
    v = f"B{k+1}"
    vehicles.append(v)
    vehicle_type[v] = "bike"
    Q[v] = 10
    F[v] = 800
    c[v] = 3
    speed[v] = 20

# 3 motorcycles
for k in range(3):
    v = f"M{k+1}"
    vehicles.append(v)
    vehicle_type[v] = "moto"
    Q[v] = 25
    F[v] = 1500
    c[v] = 10
    speed[v] = 40

V = vehicles
service_time = 3/60.0     # hours
T_max = 2.5               # hours

print("Vehicles:", V)


Vehicles: ['B1', 'B2', 'B3', 'B4', 'M1', 'M2', 'M3']


In [97]:
prob = pulp.LpProblem("Clustering_Q1", pulp.LpMinimize)

# Decision variables
z = pulp.LpVariable.dicts("z", (I, V), lowBound=0, upBound=1, cat=pulp.LpBinary)
u = pulp.LpVariable.dicts("u", V, lowBound=0, upBound=1, cat=pulp.LpBinary)

# Each customer is assigned to exactly one vehicle
for i in I:
    prob += pulp.lpSum(z[i][v] for v in V) == 1, f"assign_{i}"

# Capacity constraints
for v in V:
    prob += pulp.lpSum(q[i] * z[i][v] for i in I) <= Q[v] * u[v], f"cap_{v}"

# Link z and u
for i in I:
    for v in V:
        prob += z[i][v] <= u[v], f"link_{i}_{v}"

# Availability (not strictly needed since we defined 4+3 vehicles)
prob += pulp.lpSum(u[v] for v in V if vehicle_type[v] == "bike") <= 4, "max_bikes"
prob += pulp.lpSum(u[v] for v in V if vehicle_type[v] == "moto") <= 3, "max_motos"

# Approximate objective
prob += (
    pulp.lpSum(F[v] * u[v] for v in V) +
    pulp.lpSum(c[v] * d[("depot", i)] * z[i][v] for v in V for i in I)
)

prob.solve(pulp.PULP_CBC_CMD(msg=1))
print("Status:", pulp.LpStatus[prob.status])


Status: Optimal


In [98]:
clusters = {v: [] for v in V}
for i in I:
    for v in V:
        if pulp.value(z[i][v]) > 0.5:
            clusters[v].append(i)

print("Clusters:")
for v in V:
    print(v, "->", clusters[v])


Clusters:
B1 -> [4, 26, 35, 36, 38, 41]
B2 -> [0, 27, 29, 39, 44, 49]
B3 -> []
B4 -> [16, 18, 20, 22, 32, 37]
M1 -> [1, 5, 10, 13, 14, 15, 17, 19, 21, 23, 24, 25, 34, 43, 45, 47]
M2 -> [2, 3, 6, 7, 8, 9, 11, 12, 28, 30, 31, 33, 40, 42, 46, 48]
M3 -> []


In [99]:
def solve_tsp_cluster(Iv, d, speed_v, service_time, T_max, name="TSP"):
    if len(Iv) == 0:
        return 0.0, []

    Nv = ["depot"] + Iv
    prob = pulp.LpProblem(name, pulp.LpMinimize)

    x = pulp.LpVariable.dicts("x", (Nv, Nv), lowBound=0, upBound=1, cat=pulp.LpBinary)

    # MTZ order variables (customers only)
    nv = len(Iv)
    u = pulp.LpVariable.dicts("u", Iv, lowBound=1, upBound=nv, cat=pulp.LpContinuous)

    # Flow constraints for customers
    for i in Iv:
        prob += pulp.lpSum(x[i][j] for j in Nv if j != i) == 1, f"out_{i}"
        prob += pulp.lpSum(x[j][i] for j in Nv if j != i) == 1, f"in_{i}"

    # Depot in/out
    prob += pulp.lpSum(x["depot"][j] for j in Iv) == 1, "depot_out"
    prob += pulp.lpSum(x[j]["depot"] for j in Iv) == 1, "depot_in"

    # No self loops
    for i in Nv:
        prob += x[i][i] == 0, f"no_loop_{i}"

    # MTZ subtour elimination
    for i in Iv:
        for j in Iv:
            if i != j:
                prob += u[i] - u[j] + 1 <= nv * (1 - x[i][j]), f"mtz_{i}_{j}"

    # Route length
    route_length = pulp.lpSum(d[(i, j)] * x[i][j] for i in Nv for j in Nv)

    # Time constraint
    route_time = route_length / speed_v + service_time * len(Iv)
    prob += route_time <= T_max, "time_limit"

    # Objective: minimize route length
    prob += route_length

    prob.solve(pulp.PULP_CBC_CMD(msg=0))
    if pulp.LpStatus[prob.status] != "Optimal":
        print("Warning: TSP status:", pulp.LpStatus[prob.status])

    L_val = pulp.value(route_length)

    # Extract edges
    edges = []
    for i in Nv:
        for j in Nv:
            if pulp.value(x[i][j]) > 0.5:
                edges.append((i, j))

    return L_val, edges


In [100]:
total_cost = 0.0
vehicle_rows = []

for v in V:
    Iv = clusters[v]
    if len(Iv) == 0:
        continue

    print(f"\nVehicle {v} ({vehicle_type[v]}) with {len(Iv)} customers")
    L_v, edges_v = solve_tsp_cluster(Iv, d, speed[v], service_time, T_max, name=f"TSP_{v}")

    fixed_cost = F[v]
    variable_cost = c[v] * L_v
    cost_v = fixed_cost + variable_cost
    total_cost += cost_v

    print(f"  Route length: {L_v:.2f} km")
    print(f"  Fixed cost  : {fixed_cost:.2f}")
    print(f"  Var cost    : {variable_cost:.2f}")
    print(f"  Total (v)   : {cost_v:.2f}")

    # Simple reconstruction of route order
    succ = {i: j for (i, j) in edges_v}
    route = ["depot"]
    cur = "depot"
    while True:
        nxt = succ.get(cur, None)
        if nxt is None or nxt == "depot":
            route.append("depot")
            break
        route.append(nxt)
        cur = nxt
    print("  Route:", route)

    vehicle_rows.append({
        "vehicle": v,
        "type": vehicle_type[v],
        "num_customers": len(Iv),
        "route_length_km": L_v,
        "fixed_cost": fixed_cost,
        "variable_cost": variable_cost,
        "total_cost": cost_v
    })

print("\nTOTAL SYSTEM COST:", total_cost)



Vehicle B1 (bike) with 6 customers
  Route length: 42.53 km
  Fixed cost  : 800.00
  Var cost    : 127.60
  Total (v)   : 927.60
  Route: ['depot', 'depot']

Vehicle B2 (bike) with 6 customers
  Route length: 35.41 km
  Fixed cost  : 800.00
  Var cost    : 106.22
  Total (v)   : 906.22
  Route: ['depot', 'depot']

Vehicle B4 (bike) with 6 customers
  Route length: 41.64 km
  Fixed cost  : 800.00
  Var cost    : 124.92
  Total (v)   : 924.92
  Route: ['depot', 18, 22, 32, 20, 16, 37, 'depot']

Vehicle M1 (moto) with 16 customers
  Route length: 45.67 km
  Fixed cost  : 1500.00
  Var cost    : 456.65
  Total (v)   : 1956.65
  Route: ['depot', 14, 45, 47, 25, 17, 21, 19, 13, 43, 24, 5, 15, 1, 34, 23, 10, 'depot']

Vehicle M2 (moto) with 16 customers
  Route length: 62.79 km
  Fixed cost  : 1500.00
  Var cost    : 627.88
  Total (v)   : 2127.88
  Route: ['depot', 40, 28, 30, 46, 6, 48, 42, 3, 7, 9, 12, 11, 8, 31, 2, 33, 'depot']

TOTAL SYSTEM COST: 6843.269170222849
